In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.stats as st
import seaborn as sns
from mpmath import *
from decimal import Decimal

import tools as tl
import benchmark_func as bf

In [18]:
ml_model_folder = "./data_files/ml_models_times/"
files = os.listdir(ml_model_folder)
files = [file for file in files if 'DS_S' not in file]

In [15]:
dimensions = list(set(file.split('-')[1] for file in files))

problems = list(set(file.split('-')[0] for file in files))
len(problems), dimensions

(107, ['50D', '2D', '10D', '30D'])

In [16]:
file_label = 'default_nn_best_double_lstm'
pred_dyn_time = '_mhs_dynamic_time_logs'
pred_nn_time = '_mhs_time_prediction_logs'
nn_train_time = '-mem60_log_time'
nn_train_epoch = '-mem60_log'


In [33]:
time_training = []
for problem in problems:
  for dimension in dimensions:
    kk = ml_model_folder+'-'.join([problem, dimension, file_label+nn_train_time])+'.csv'
    df = pd.read_csv(kk,index_col=0)
    time_training.append(sum(df['time'])/60)
      

In [34]:
np.mean(time_training)

22.065762639339116

In [51]:
print(f"{np.mean(time_training):.4f} +- {np.std(time_training):.4f} min")

22.0658 +- 1.1422 min


In [35]:
np.max(time_training)

26.4590216199557

In [36]:
np.min(time_training)

18.90218506924187

In [38]:
np.median(time_training)

22.254574124080442

In [74]:
prediction_nn = []
prediction_dyn = []
model_size = []
for problem in problems:
  for dimension in dimensions:
    kk = ml_model_folder+'-'.join([problem, dimension, file_label+pred_nn_time])+'.csv'
    df = pd.read_csv(kk,index_col=0)
    prediction_nn.append(list(df['time']))

    kk = ml_model_folder+'-'.join([problem, dimension, file_label+pred_dyn_time])+'.csv'
    df = pd.read_csv(kk,index_col=0)
    prediction_dyn.append(list(df['time']))
    
    kk = ml_model_folder+'-'.join([problem, dimension, file_label])+'.h5'
    szmb = os.path.getsize(kk)
    model_size.append(szmb)
    
    
prediction_nn = sum(prediction_nn, [])
prediction_dyn = sum(prediction_dyn, [])


In [77]:

kk = tl.read_json('dlstm_cec2005_results.json')
application_dsltm = kk['time']

In [75]:

kk = tl.read_json('bmhs_cec2005_results.json')
generation_bmhs = kk['time']

In [66]:
def statistics(arr):
  print(f'{np.mean(arr):.4f} +- {np.std(arr):.4f}')
  print("Mean:", np.mean(arr))
  print("Median:", np.median(arr))
  print("Max:", np.max(arr))
  print("Min:", np.min(arr))

In [80]:
statistics(np.array(application_dsltm)/30)

0.1150 +- 0.1330
Mean: 0.11503951610396822
Median: 0.08288715346666702
Max: 0.9077741125000007
Min: 0.0028204264000000496


In [76]:
statistics(generation_bmhs)

7.1207 +- 9.8421
Mean: 7.120677338597865
Median: 3.9960812079998504
Max: 75.5595616669998
Min: 1.0285562089998166


In [67]:
statistics(prediction_dyn)

0.7847 +- 0.4637
Mean: 0.7847310644040911
Median: 0.685302846133709
Max: 6.7967276107519865
Min: 0.1729795932769775


In [68]:
statistics(prediction_nn)

2.8644 +- 1.7053
Mean: 2.864399075801702
Median: 3.0174137512221932
Max: 12.9883802421391
Min: 0.1112331077456474


In [65]:
statistics(model_size) # In bytes

155844.0 +- 0.0
Mean: 155844.0
Median: 155844.0
Max: 155844
Min: 155844


In [53]:
from model_profiler import model_profiler

import tensorflow as tf

In [54]:
path = "./data_files/ml_models/Ackley1-2D-default_nn_best_double_lstm.h5"

In [55]:
model = tf.keras.models.load_model(path)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-10-10 01:26:58.555176: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-10 01:26:58.555376: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [56]:
profile = model_profiler(model, 32)
print(profile)

| Model Profile                    | Value   | Unit    |
|----------------------------------|---------|---------|
| Selected GPUs                    | ['0']   | GPU IDs |
| No. of FLOPs                     | 0.0     | BFLOPs  |
| GPU Memory Requirement           | 0.0002  | GB      |
| Model Parameters                 | 0.0093  | Million |
| Memory Required by Model Weights | 0.0356  | MB      |


2022-10-10 01:28:24.913328: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-10 01:28:24.913386: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/josetapia/miniforge3/envs/tensorflow/lib/python3.9/site-packages/model_profiler/profiler.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.asarray(values).reshape(-1,1),


In [57]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 20)            1760      
                                                                 
 lstm_1 (LSTM)               (None, 20)                3280      
                                                                 
 dense (Dense)               (None, 205)               4305      
                                                                 
Total params: 9,345
Trainable params: 9,345
Non-trainable params: 0
_________________________________________________________________


In [60]:
def get_model_memory_usage(batch_size, model):
    import numpy as np
    try:
        from keras import backend as K
    except:
        from tensorflow.keras import backend as K

    shapes_mem_count = 0
    internal_model_mem_count = 0
    for l in model.layers:
        layer_type = l.__class__.__name__
        if layer_type == 'Model':
            internal_model_mem_count += get_model_memory_usage(batch_size, l)
        single_layer_mem = 1
        out_shape = l.output_shape
        if type(out_shape) is list:
            out_shape = out_shape[0]
        for s in out_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in model.trainable_weights])
    non_trainable_count = np.sum([K.count_params(p) for p in model.non_trainable_weights])

    number_size = 4.0
    if K.floatx() == 'float16':
        number_size = 2.0
    if K.floatx() == 'float64':
        number_size = 8.0

    total_memory = number_size * (batch_size * shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 10) + internal_model_mem_count
    return gbytes

In [61]:
get_model_memory_usage(32, model)

0.0002046861